In [1]:
from gensim import corpora, models
import json
from nltk.corpus import stopwords
import re
from pyLDAvis import gensim, display

/Users/jcapde87/Envs/udg_seminar/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
# WITHOUT AGGREGATION

In [3]:
data_file = "data/tweets.json"
text = []
lang = []
with open(data_file,"r") as json_file:
    for line in json_file:
        tweet = json.loads(line)
        lang.append(tweet["lang"])
        text.append(tweet["text"])
    

In [4]:
stopwords.words('english')

toIgnore_expr = [r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # url
                 r'&[a-z]+;' # html entities
                 ]
all_toIgnore = '|'.join(toIgnore_expr)

def remove_waste(text):
    return re.sub(all_toIgnore," ",text)

def tokenize(sentence):
    stopw = stopwords.words('spanish') + [u'si', u'barcelona'] +stopwords.words('english')
    return [word for word in re.findall(r"[#|@|A-Z|a-z]\w*", remove_waste(sentence))
                if len(word)>1 and word[0]!="#" and word[0]!="@" and word.lower() not in stopw]


In [5]:
docs = [tokenize(txt) for i, txt in enumerate(text)]

In [7]:
dictionary = corpora.Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]
K = 30

In [9]:
lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=K, passes=10)

In [10]:
vis_data = gensim.prepare(lda, corpus, dictionary)
display(vis_data)

In [ ]:
# AGGREGATING BY HASHTAG
data_file = "data/tweets.json"

hts = []
hts_l = []
text = []
lang = []

with open(data_file,"r") as json_file:
    for line in json_file:
        tweet = json.loads(line)
        if len(tweet["entities"]["hashtags"])>0:
            aux = [ht['text'] for ht in tweet["entities"]["hashtags"]]
            hts += aux
            hts_l.append(aux)
            text += [tweet["text"]]
            lang += tweet["lang"]

unique_hts = list(set(hts))


In [17]:
doc_ht = []
for ht in unique_hts:
    indices = [i for i, x in enumerate(hts_l) for h in x if h == ht]
    doc_ht += [" ".join([text[i] for i in indices])]

In [18]:
texts = [tokenize(txt) for i, txt in enumerate(doc_ht)]

In [19]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
K = 30

In [20]:
lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=K, passes=10)

In [21]:
vis_data = gensim.prepare(lda, corpus, dictionary)
display(vis_data)